In [74]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Concatenate, Flatten, Dense
from sklearn import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Concatenate, Flatten, Dense, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
"""
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
"""
import warnings
warnings.filterwarnings('ignore')

# Reset display options to default
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

## DATOS A SABER:
97 skills en dataframe train

174 alumnos

12615 combinaciones Unit-Sect-Prob-Step

In [25]:
# Leer el DataFrame (ajusta el archivo CSV según tu ubicación)
train_csv = pd.read_csv('CSVs/Clean_Train.csv')
train_csv=train_csv[train_csv['Problem View'] <= 8]

In [26]:
# 2. Crear la combinación de características y tokenizar
train_csv['Combination'] = train_csv['Unit'] + "_" + train_csv['Section'] + "_" + train_csv['Problem Name'] + "_" + train_csv['Step Name']

In [27]:
print(len(train_csv['Anon Student Id'].unique()))
print(len(train_csv['Combination'].unique()))


174
12607


### NORMALIZACIÓN RENDIMIENTO Y DIFICULTAD

In [28]:
train_csv['Dificultad'] = train_csv['Dificultad'] / 10.0
train_csv['Rendimiento'] = train_csv['Rendimiento'] / 5.0

### ESTANDARIZACIÓN OPORTUNIDADES

In [29]:
opportunity_columns = ['Opportunity_1','Opportunity_2','Opportunity_3','Opportunity_4','Opportunity_5']
scaler = preprocessing.StandardScaler()
# Unir las columnas de oportunidad en una sola matriz para el ajuste
opportunity_data = train_csv[opportunity_columns].values.flatten().reshape(-1, 1)
scaler.fit(opportunity_data)

# Aplicar la transformación a cada columna de oportunidades
for col in opportunity_columns:
    train_csv[col] = scaler.transform(train_csv[col].values.reshape(-1, 1))


In [30]:
train_csv

,Unnamed: 0,Anon Student Id,Unit,Section,Problem Name,Problem View,Step Name,Correct First Attempt,Dificultad,Rendimiento,...,KC_2,KC_3,KC_4,KC_5,Opportunity_1,Opportunity_2,Opportunity_3,Opportunity_4,Opportunity_5,Combination
0,0,52vEY7f17k,CTA1_13,CTA1_13-1,PROP04,1,R1C1,1,0.2,0.4,...,0,0,0,0,-0.279050,-0.27905,-0.279050,-0.27905,-0.27905,CTA1_13_CTA1_13-1_PROP04_R1C1
1,1,52vEY7f17k,CTA1_13,CTA1_13-1,PROP04,1,R1C2,1,0.3,0.4,...,0,0,0,0,-0.279050,-0.27905,-0.279050,-0.27905,-0.27905,CTA1_13_CTA1_13-1_PROP04_R1C2
2,2,52vEY7f17k,CTA1_13,CTA1_13-1,PROP04,1,R3C1,0,0.2,0.4,...,0,0,0,0,-0.261930,-0.27905,-0.279050,-0.27905,-0.27905,CTA1_13_CTA1_13-1_PROP04_R3C1
3,3,52vEY7f17k,CTA1_13,CTA1_13-1,PROP04,1,R3C2,0,0.7,0.4,...,"Write_expression,_positive_slope",Using_difficult_numbers,0,0,-0.261930,-0.26193,-0.261930,-0.27905,-0.27905,CTA1_13_CTA1_13-1_PROP04_R3C2
4,4,52vEY7f17k,CTA1_13,CTA1_13-1,PROP04,1,R4C1,1,0.4,0.4,...,0,0,0,0,-0.261930,-0.27905,-0.279050,-0.27905,-0.27905,CTA1_13_CTA1_13-1_PROP04_R4C1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202866,246701,JP20cryBg8,CTA1_24,CTA1_24-1,NOV05,1,R5C1,0,0.7,0.2,...,"Convert_unit,_mixed",0,0,0,0.080471,-0.26193,-0.279050,-0.27905,-0.27905,CTA1_24_CTA1_24-1_NOV05_R5C1
202867,246702,JP20cryBg8,CTA1_24,CTA1_24-1,NOV05,1,R1C2,1,0.1,0.2,...,0,0,0,0,-0.279050,-0.27905,-0.279050,-0.27905,-0.27905,CTA1_24_CTA1_24-1_NOV05_R1C2
202868,246703,JP20cryBg8,CTA1_24,CTA1_24-1,NOV05,1,R2C2,1,0.1,0.2,...,0,0,0,0,-0.159210,-0.27905,-0.279050,-0.27905,-0.27905,CTA1_24_CTA1_24-1_NOV05_R2C2
202869,246704,JP20cryBg8,CTA1_24,CTA1_24-1,NOV05,1,R3C2,0,0.2,0.2,...,"Write_expression,_positive_slope",Using_difficult_numbers,0,0,-0.142090,-0.19345,-0.090729,-0.27905,-0.27905,CTA1_24_CTA1_24-1_NOV05_R3C2


In [31]:
# 1. Tokenización del ID del Alumno
# 1. Tokenización del ID del Alumno
student_id_encoder = LabelEncoder()
train_csv['Student ID Token'] = student_id_encoder.fit_transform(train_csv['Anon Student Id'])

# 2. Crear la combinación de características y tokenizar
train_csv['Combination'] = train_csv['Unit'] + "_" + train_csv['Section'] + "_" + train_csv['Problem Name'] + "_" + train_csv['Step Name']

# Verificar el número de combinaciones únicas
num_combinations_unique = len(train_csv['Combination'].unique())
print("Número de combinaciones únicas:", num_combinations_unique)

# Usar LabelEncoder en lugar de Tokenizer para la combinación
combination_encoder = LabelEncoder()
train_csv['Combination Token'] = combination_encoder.fit_transform(train_csv['Combination'])

# Verificar el tamaño del vocabulario tokenizado
num_combinations_tokenized = len(combination_encoder.classes_)
print("Tamaño del vocabulario tokenizado:", num_combinations_tokenized)

# Tokenizar cada KC
kc_columns = ['KC_1', 'KC_2', 'KC_3', 'KC_4', 'KC_5']
for kc in kc_columns:
    kc_encoder = LabelEncoder()
    train_csv[kc + ' Token'] = kc_encoder.fit_transform(train_csv[kc].astype(str))  # Ensure data type is string
    kc_vocab_size = len(train_csv[kc + ' Token'].unique())
    print(f"Tokenized vocabulary size for {kc}: {kc_vocab_size}")


# 4. Preparar los datos para el modelo
X_student_id = train_csv['Student ID Token'].values
X_combination = train_csv['Combination Token'].values

# Preparar los datos de KC para el modelo
X_kc = train_csv[[f'{kc} Token' for kc in kc_columns]].values






Número de combinaciones únicas: 12607
Tamaño del vocabulario tokenizado: 12607
Tokenized vocabulary size for KC_1: 92
Tokenized vocabulary size for KC_2: 45
Tokenized vocabulary size for KC_3: 36
Tokenized vocabulary size for KC_4: 16
Tokenized vocabulary size for KC_5: 5
Número de estudiantes únicos: 174
Número de combinaciones únicas: 12607
Número de KC únicos por cada KC: [92, 45, 36, 16, 5]


In [32]:
print(num_students)
print(num_combinations)
print(num_kcs)

174
12607
[92, 45, 36, 16, 5]


In [37]:
print(X_kc[3])

[69 36 17  0  0]


In [71]:
# Ajustar números de clases y dimensión de embedding según los datos
num_students = len(student_id_encoder.classes_)
num_combinations = len(combination_encoder.classes_)
num_kcs = [len(train_csv[f'{kc} Token'].unique()) for kc in kc_columns]

"""
print("Número de estudiantes únicos:", num_students)
print("Número de combinaciones únicas:", num_combinations)
print("Número de KC únicos por cada KC:", num_kcs)
"""

embedding_dim_student = 16
embedding_dim_combination = 64
embedding_dim_kc = 16

# 5. Construcción del Modelo
input_student_id = Input(shape=(1,), dtype='int32', name='student_id')
embedding_student = Embedding(input_dim=num_students, output_dim=embedding_dim_student)(input_student_id)

input_combination = Input(shape=(1, ), dtype='int32', name='U.S.P.S')
embedding_combination = Embedding(input_dim=num_combinations, output_dim=embedding_dim_combination)(input_combination)

input_kcs = Input(shape=(5,), dtype='int32', name='KCs')
embedding_kcs = Embedding(input_dim=100, output_dim=embedding_dim_kc)(input_kcs)

input_op_pv_rd = Input(shape=(8,), dtype='float32', name='op_pv_rd')

# Concatenar todas las representaciones
concatenated = Concatenate()([Flatten()(embedding_student), Flatten()(embedding_combination), Flatten()(embedding_kcs), Flatten()(input_op_pv_rd)])

dense = Dense(256, activation='relu')(concatenated)
dense = BatchNormalization()(dense)
dense = Dropout(0.5)(dense)

dense = Dense(64, activation='relu')(concatenated)
dense = BatchNormalization()(dense)
dense = Dropout(0.5)(dense)

dense = Dense(32, activation='relu')(dense)
dense = BatchNormalization()(dense)
dense = Dropout(0.5)(dense)

output = Dense(1, activation='sigmoid')(dense)  # Para una tarea de clasificación binaria

# Definir el modelo
model = Model(inputs=[input_student_id, input_combination, input_kcs, input_op_pv_rd], outputs=output)

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()


Model: "functional_30"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ student_id          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ U.S.P.S             │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ KCs (InputLayer)    │ (None, 5)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_63        │ (None, 1, 16)     │      2,784 │ student_id[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_64        │ (None, 1, 64)     │    806,848 │ U.S.P.S[0][0]     │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_65        │ (None, 5, 16)     │      1,600 │ KCs[0][0]         │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ op_pv_rd            │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_64          │ (None, 16)        │          0 │ embedding_63[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_65          │ (None, 64)        │          0 │ embedding_64[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_66          │ (None, 80)        │          0 │ embedding_65[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_67          │ (None, 8)         │          0 │ op_pv_rd[0][0]    │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_16      │ (None, 168)       │          0 │ flatten_64[0][0], │
│ (Concatenate)       │                   │            │ flatten_65[0][0], │
│                     │                   │            │ flatten_66[0][0], │
│                     │                   │            │ flatten_67[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_57 (Dense)    │ (None, 64)        │     10,816 │ concatenate_16[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_57[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_26          │ (None, 64)        │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_58 (Dense)    │ (None, 32)        │      2,080 │ dropout_26[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32)        │        128 │ dense_58[0][0]    │
│ (BatchNormalizatio… │                   │            │                 

 Total params: 824,545 (3.15 MB)

 Trainable params: 824,353 (3.14 MB)

 Non-trainable params: 192 (768.00 B)

In [72]:
y = train_csv['Correct First Attempt'].values

# Preparar las características adicionales para el modelo
X_op_pv_rd = train_csv[['Opportunity_1', 'Opportunity_2', 'Opportunity_3', 'Opportunity_4', 'Opportunity_5', 'Problem View', 'Dificultad', 'Rendimiento']].values

# Ajustar el modelo con tus datos
model.fit([X_student_id, X_combination, X_kc, X_op_pv_rd], y, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
5010/5010 ━━━━━━━━━━━━━━━━━━━━ 36s 7ms/step - accuracy: 0.7662 - loss: 0.5041 - val_accuracy: 0.8357 - val_loss: 0.3806
Epoch 2/10
5010/5010 ━━━━━━━━━━━━━━━━━━━━ 34s 7ms/step - accuracy: 0.8213 - loss: 0.4052 - val_accuracy: 0.8352 - val_loss: 0.3826
Epoch 3/10
5010/5010 ━━━━━━━━━━━━━━━━━━━━ 34s 7ms/step - accuracy: 0.8252 - loss: 0.3965 - val_accuracy: 0.8353 - val_loss: 0.3853
Epoch 4/10
5010/5010 ━━━━━━━━━━━━━━━━━━━━ 34s 7ms/step - accuracy: 0.8292 - loss: 0.3887 - val_accuracy: 0.8339 - val_loss: 0.3891
Epoch 5/10
5010/5010 ━━━━━━━━━━━━━━━━━━━━ 34s 7ms/step - accuracy: 0.8318 - loss: 0.3860 - val_accuracy: 0.8349 - val_loss: 0.3902
Epoch 6/10
5010/5010 ━━━━━━━━━━━━━━━━━━━━ 34s 7ms/step - accuracy: 0.8309 - loss: 0.3851 - val_accuracy: 0.8338 - val_loss: 0.3939
Epoch 7/10
5010/5010 ━━━━━━━━━━━━━━━━━━━━ 37s 7ms/step - accuracy: 0.8320 - loss: 0.3815 - val_accuracy: 0.8321 - val_loss: 0.3912
Epoch 8/10
5010/5010 ━━━━━━━━━━━━━━━━━━━━ 37s 7ms/step - accuracy: 0.8347 - loss: 0

In [73]:
# Ajustar números de clases y dimensión de embedding según los datos
num_students = len(student_id_encoder.classes_)
num_combinations = len(combination_encoder.classes_)
num_kcs = [len(train_csv[f'{kc} Token'].unique()) for kc in kc_columns]

"""
print("Número de estudiantes únicos:", num_students)
print("Número de combinaciones únicas:", num_combinations)
print("Número de KC únicos por cada KC:", num_kcs)
"""

embedding_dim_student = 16
embedding_dim_combination = 64
embedding_dim_kc = 16

# 5. Construcción del Modelo
input_student_id = Input(shape=(1,), dtype='int32', name='student_id')
embedding_student = Embedding(input_dim=num_students, output_dim=embedding_dim_student)(input_student_id)

input_combination = Input(shape=(1, ), dtype='int32', name='U.S.P.S')
embedding_combination = Embedding(input_dim=num_combinations, output_dim=embedding_dim_combination)(input_combination)

input_kcs = Input(shape=(5,), dtype='int32', name='KCs')
embedding_kcs = Embedding(input_dim=100, output_dim=embedding_dim_kc)(input_kcs)

input_op_pv_rd = Input(shape=(8,), dtype='float32', name='op_pv_rd')

# Concatenar todas las representaciones
concatenated = Concatenate()([Flatten()(embedding_student), Flatten()(embedding_combination), Flatten()(embedding_kcs), Flatten()(input_op_pv_rd)])
model2 = Model(inputs=[input_student_id, input_combination, input_kcs, input_op_pv_rd], outputs=concatenated)


In [75]:
# 6. Obtener la salida del modelo para usarla como entrada al SVM
embedding_model = Model(inputs=model.inputs, outputs=model.output)

# Preparar las características adicionales para el modelo
X_op_pv_rd = train_csv[['Opportunity_1', 'Opportunity_2', 'Opportunity_3', 'Opportunity_4', 'Opportunity_5', 'Problem View', 'Dificultad', 'Rendimiento']].values

# Generar las embeddings para todos los datos
X_embeddings = embedding_model.predict([X_student_id, X_combination, X_kc, X_op_pv_rd])

# Etiquetas
y = train_csv['Correct First Attempt'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_embeddings, y, test_size=0.2, random_state=42)

# 7. Entrenamiento del modelo SVM
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train, y_train)

# Predicción y evaluación
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

6262/6262 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step


In [76]:
print(accuracy)

0.8415011478191436
